# Proteomics Integration

Load a enzyme constrained metabolic mdoel of _Escherichia coli_.

In [2]:
import pandas as pd

from cobra.io import read_sbml_model
from cobra.flux_analysis import flux_variability_analysis

model = read_sbml_model('data/eciML1515.xml.gz')

The model has two differences with a standard COBRA model. First, the reactions contain another _metabolite_: the enyzme itself.

In [4]:
model.reactions.FRD2No1

Reaction identifier,FRD2No1
Name,Fumarate reductase (No1)
Memory address,0x07fedae3534e0
Stoichiometry,fum_c + mql8_c + 1.1111e-06 prot_P00363 + 1.1111e-06 prot_P0A8Q0 + 1.1111e-06 prot_P0A8Q3 + 1.1111e-06 prot_P0AC47 --> mqn8_c + succ_c Fumarate [cytosol] + Menaquinol 8 [cytosol] + 1.1111e-06 prot_P00363 [cytosol] + 1.1111e-06 prot_P0A8Q0 [cytosol] + 1.1111e-06 prot_P0A8Q3 [cytosol] + 1.1111e-06 prot_P0AC47 [cytosol] --> Menaquino...
GPR,b4153 and b4152 and b4151 and b4154
Lower bound,0.0
Upper bound,1000.0


In this model, all protein ids follow the form `prot_UNIPROT`.

The second difference is the existence of _protein exchange reactions_. These protein exchanges follow the naming `prot_UNIPROT_exchange`.

In [5]:
model.reactions.prot_P00363_exchange

Reaction identifier,prot_P00363_exchange
Name,prot_P00363_exchange
Memory address,0x07fedacdccfd0
Stoichiometry,--> prot_P00363 --> prot_P00363 [cytosol]
GPR,b4154
Lower bound,0.0
Upper bound,1000.0


By putting an upper bound on these exchanges, we will integrate proteomics data into the model and treat it as an usual COBRA model without further changes.

## 1. Proteomics data preparation
Proteomics data is usually presented as Number of copies per cell. We need to do convert this information into the units used by the enzyme constrained model.

<div class="alert alert-block alert-warning">
<b>DEBUG:</b> Should run PCA for the entire dataset?
</div>

In [14]:
df = pd.read_csv(
    #"data/ecoli_proteomics_schmidt2016_S9.tsv", # our strain but just Glc/LB
    "data/ecoli_proteomics_schmidt2016_S5_ren.tsv", # BW25113 22 media
    "\t", skiprows=2  # skip titles and subtitles (XLXS)
)
exp_details = pd.read_csv(
    "data/ecoli_details_schmidt2016_S23.tsv", "\t", 
    skiprows=2  # skip titles and subtitles (XLXS)
)

In [15]:
df.head()

,Uniprot Accession,Description,Gene,Peptides used for quantitation,Confidence score,Molecular weight (Da),Glucose_copies,LB_copies,Glycerol + AA_copies,Acetate_copies,...,Stationary phase 1 day_cv,Stationary phase 3 days_cv,Osmotic-stress glucose_cv,42°C glucose_cv,pH6 glucose_cv,Xylose_cv,Mannose_cv,Galactose _cv,Succinate_cv,Fructose_cv
0,P0A8T7,DNA-directed RNA polymerase subunit beta' OS=E...,rpoC,91,6045.53,155045.0080,2779,7164,4503,2180,...,6.76,12.08,20.91,13.86,6.14,6.27,16.15,22.87,16.45,9.29
1,P0A8V2,DNA-directed RNA polymerase subunit beta OS=Es...,rpoB,89,5061.29,150520.2758,3957,8888,5199,2661,...,14.42,11.18,17.48,10.51,5.93,4.27,13.51,19.75,13.60,7.77
2,P36683,Aconitate hydratase 2 OS=Escherichia coli (str...,acnB,67,4505.67,93420.9457,7596,16600,17548,22844,...,3.93,10.04,16.60,9.57,4.47,21.78,3.16,2.07,4.21,3.25
3,P15254,Phosphoribosylformylglycinamidine synthase OS=...,purL,65,4277.71,141295.8984,2456,821,2339,1438,...,7.01,4.02,12.97,11.38,1.78,7.88,1.22,7.69,2.96,6.20
4,P09831,Glutamate synthase [NADPH] large chain OS=Esch...,gltB,64,4111.74,163176.3153,2859,604,652,1363,...,17.44,3.98,28.41,17.39,7.84,20.05,17.71,22.36,10.69,8.91


In [16]:
exp_details.head()

,Growth condition,Strain,Growth rate (h-1),Stdev,Single cell volume [fl]1,Doubling time (h-1),Time exp before harvest (h),# of doublings at exponential growth before harvesting,OD @ harvesting. replicates,Unnamed: 9,Unnamed: 10,Number of Proteins Identified (FDR 1%)2
0,LB,BW25113,1.90,0.03,4.29,0.4,5,13.7,1.80,1.37,1.34,1752.0
1,LB,MG1665,1.78,0.05,4.23,0.4,5.4,13.9,0.53,1.09,1.32,1733.0
2,LB,NCM3722,2.30,0.05,4.36,0.3,3.2,10.6,0.91,0.96,1.00,1703.0
3,Glycerol + AA,BW25113,1.27,0.01,3.83,0.5,7.5,13.8,0.50,0.51,0.50,1675.0
4,Acetate,BW25113,0.30,0.04,2.30,2.3,29.3,12.7,0.53,0.22,0.48,1683.0


In [17]:
df = df.loc[:, 
    df.columns.str.contains("_copies|_cv", regex=True) |  # only interested in copies/cell and uncertainty
    df.columns.isin(["Uniprot Accession"])  # and relevant info about proteins
]

In [18]:
df_ac = df.loc[:, ["Uniprot Accession", "Acetate_copies", "Acetate_cv"]]
# rename resulting columns
df_ac.columns = ["uniprot", "copies_per_cell", "CV"]

In [19]:
df_ac.describe()

,copies_per_cell,CV
count,2058.000000,2058.000000
mean,1847.405248,17.656040
std,8487.943632,20.773887
min,0.000000,0.190000
25%,29.000000,6.680000
50%,170.000000,11.425000
75%,868.250000,19.892500
max,242737.000000,170.100000


In [20]:
# apply uncertainty (extend upper bound as 1/2 of stdev)
df_ac["copies_upper"] = df_ac["copies_per_cell"] + 0.5 * df_ac["CV"]/100 * df_ac["copies_per_cell"]

First, convert copies per cell to abundance per cell.
\begin{align}
\frac{\text{mmol}}{\text{cell}} = \frac{\text{molecules}}{\text{cell}} \frac{10^3\text{mol}}{\text{molecules}}
\end{align}

In [21]:
df_ac["mmol_per_cell"] = df_ac["copies_upper"] * 1e3/6.022e23

And, then, convert the abundance per cell into abundance per gDW.

\begin{align}
\frac{\text{mmol}}{\text{gDW}} = \frac{\text{mmol}}{\text{cell}} \frac{\text{cell}}{fL} \frac{fL}{g}\frac{g}{\text{gDW}}
\end{align}

In [22]:
growth_experimental = exp_details.loc[
    (exp_details["Growth condition"]=="Acetate") & (exp_details["Strain"]=="BW25113"), 
    "Growth rate (h-1)"
].values[0]
cell_volume = exp_details.loc[
    (exp_details["Growth condition"]=="Acetate") & (exp_details["Strain"]=="BW25113"), 
    "Single cell volume [fl]1"
].values[0]
cell_density = 1.105e-12
water_content = 0.3

In [23]:
df_ac["conc"] = df_ac["mmol_per_cell"] * 1 / (cell_volume * cell_density * water_content)

In [24]:
proteomics = df_ac["conc"]
proteomics.index = df_ac["uniprot"]

## 2. Model
Simulations part (caffeine)

In [25]:
# save the non enzyme-constrained model
plain_model = model.copy()

In [26]:
def limit_proteins(model, measurements):
    """Apply proteomics measurements to `model`.

    Adapted from https://github.com/DD-DeCaF/simulations/blob/devel/src/simulations/modeling/driven.py

    Parameters
    ----------
    model: cobra.Model
        The enzyme-constrained model.
    measurements : pd.DataFrame
        Protein abundances in mmol / gDW.

    """
    for protein_id, measure in measurements.items():
        try:
            rxn = model.reactions.get_by_id(f"prot_{protein_id}_exchange")
        except KeyError:
            pass
        else:
            # update only upper_bound (as enzymes can be unsaturated):
            rxn.bounds = (0, measure)

Optimize the enzyme constrained model.

In [27]:
limit_proteins(model, proteomics)

In [29]:
# enzyme-contrained (the model doesn't grow)

model.optimize()

,fluxes,reduced_costs
EX_acgam_e,-0.000000,0.0
EX_cellb_e,-0.000000,0.0
EX_chol_e,-0.000000,0.0
EX_pi_e,0.000000,0.0
EX_h_e,6.682729,0.0
...,...,...
prot_Q59385_exchange,0.000000,0.0
prot_Q6BEX0_exchange,0.000000,0.0
prot_Q6BF16_exchange,0.000000,0.0
prot_Q6BF17_exchange,0.000000,0.0


The model can't grow!

## 3. Flexibilization

Experimental measurements can be too restrictive if an uncertainty is not given. Thus, a flexibilization of the proteomics data is usually required to work with enzyme constrained models.

In [30]:
def top_shadow_prices(solution, met_ids, top=1):
    """
    Retrieves shadow prices for a list of metabolites from the solution and ranks
    them from most to least sensitive in the model.

    Parameters
    ----------
    solution: cobra.Solution
        The usual Solution object returned by model.optimize().
    biomass_reaction: str
        name of biomass reaction
    met_ids: iterable of strings
        Subset of metabolite IDs from the model.
    top: int
        The number of metabolites to be returned.

    Returns
    -------
    shadow_pr: pd.Series
        Top shadow prices, ranked.
    """
    shadow_pr = solution.shadow_prices
    shadow_pr = shadow_pr.loc[shadow_pr.index.isin(met_ids)]
    return shadow_pr.sort_values()[:top]


def flexibilize_proteomics(
    model, biomass_reaction, minimal_growth, proteomics
):
    """
    Replace proteomics measurements with a set that enables the model to grow. Proteins
    are removed from the set iteratively based on sensitivity analysis (shadow prices).
    
    Adapted from https://github.com/DD-DeCaF/simulations/blob/devel/src/simulations/modeling/driven.py

    Parameters
    ----------
    model: cobra.Model
        The enzyme-constrained model.
    minimal_growth_rate: float
        Minimal growth rate to enforce.
    proteomics: pandas.DataFrame
        List of measurements.

    Returns
    -------
    growth_rate: dict
        New growth rate (will change if the model couldn't grow at the inputted value).
    proteomics: list(dict)
        Filtered list of proteomics.

    """
    def protein_to_metabolite(protein_id, model):
        met_id = model.metabolites.query(lambda m: protein_id in m.id)
        return met_id[0].id if met_id else ""
    
    # reset growth rate in model:
    model.reactions.get_by_id(biomass_reaction).bounds = (0, 1000)

    # build a table with protein ids, met ids in model and values to constrain with:
    prot_df = pd.DataFrame(proteomics)
    prot_df.index = prot_df.index.astype("str")
    prot_df["met_id"] = [protein_to_metabolite(prot, model) for prot in prot_df.index]
    prot_df = prot_df[prot_df.met_id != ""]
    
    # constrain the model with all proteins and optimize:
    limit_proteins(model, proteomics)
    solution = model.optimize()
    new_growth_rate = solution.objective_value if solution.objective_value else 0
    
    # relax growth constraint
    minimal_growth *= 1.05

    # while the model cannot grow to the desired level, remove the protein with
    # the highest shadow price:
    prots_to_remove = []
    while new_growth_rate < minimal_growth and not prot_df.empty:
        # get most influential protein in model:
        top_protein = top_shadow_prices(solution, list(prot_df["met_id"]))
        top_protein = top_protein.index[0]
        top_protein = prot_df.index[prot_df["met_id"] == top_protein][0]

        # update data: append protein to list, remove from current dataframe and
        # increase the corresponding upper bound to +1000:
        prots_to_remove.append(top_protein)
        prot_df = prot_df.drop(labels=top_protein)
        limit_proteins(model, pd.Series(data=[1000], index=[top_protein]))

        # re-compute solution:
        solution = model.optimize()
        #if solution.objective_value == new_growth_rate:  # the algorithm is stuck
        #    break
        new_growth_rate = solution.objective_value if solution.objective_value else 0

    # update growth rate if optimization was not successful:
    if new_growth_rate < minimal_growth:
        print(
            f"Minimal growth was not reached! "
            f"Final growth of the model: {new_growth_rate}"
        )

    return new_growth_rate, prots_to_remove

Enforce 0.1 of growth rate

In [31]:
biomass_reaction = "BIOMASS_Ec_iML1515_core_75p37M"
new_growth_rate, prots_removed = flexibilize_proteomics(model, biomass_reaction, growth_experimental, proteomics)

In [32]:
print(f"Proteins in dataset: {proteomics.shape[0]}\nProteins removed: {len(prots_removed)}")

Proteins in dataset: 2058
Proteins removed: 90


In [33]:
model.optimize()

,fluxes,reduced_costs
EX_acgam_e,0.000000,-0.162421
EX_cellb_e,0.000000,-0.121816
EX_chol_e,0.000000,0.000000
EX_pi_e,0.000000,0.000000
EX_h_e,17.879146,0.000000
...,...,...
prot_Q59385_exchange,0.000000,0.000000
prot_Q6BEX0_exchange,0.000000,0.000000
prot_Q6BF16_exchange,0.000000,0.000000
prot_Q6BF17_exchange,0.000000,0.000000


In [34]:
plain_model.optimize()

,fluxes,reduced_costs
EX_acgam_e,0.000000,-0.238866
EX_cellb_e,0.000000,-0.361121
EX_chol_e,0.000000,-0.022570
EX_pi_e,0.000000,0.000000
EX_h_e,8.058201,0.000000
...,...,...
prot_Q59385_exchange,0.000000,0.000000
prot_Q6BEX0_exchange,0.000000,0.000000
prot_Q6BF16_exchange,0.000000,0.000000
prot_Q6BF17_exchange,0.000000,0.000000


Let's compare the carbon source utilization of both models

In [35]:
# exchanges in this are on the right-had side
plain_exchanges = [reaction for reaction in plain_model.exchanges if reaction.flux > 0]
enzyme_exchanges = [reaction for reaction in model.exchanges if reaction.flux > 0]

In [36]:
plain_fva = flux_variability_analysis(plain_model)

In [37]:
enzyme_fva = flux_variability_analysis(model)

cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


In [38]:
enzyme_fva[enzyme_fva.maximum < 700].sort_values("maximum", ascending=False)

,minimum,maximum
MDH_REVNo1,0.000000,2.402620e+02
MDHNo1,0.000000,2.339076e+02
PGK_REVNo1,18.914082,1.221030e+02
PUNP5No2,0.000000,1.105349e+02
PGKNo1,0.000000,1.029215e+02
...,...,...
GUAt2pp,0.000000,-2.391933e-08
GLYt2pp,0.000000,-2.391991e-08
FORt2pp,0.000000,-2.395550e-08
ALAt4pp,0.000000,-2.403159e-08


In [39]:
plain_fva[plain_fva.maximum < 700].sort_values("maximum", ascending=False)

,minimum,maximum
prot_P21177_exchange,5.835889e-07,5.658190e+02
prot_P0AB71_exchange,0.000000e+00,3.695348e+02
prot_P0A991_exchange,0.000000e+00,1.389096e+02
prot_P33025_exchange,0.000000e+00,6.669400e+01
EX_h2o_e,4.716230e+01,5.119140e+01
...,...,...
FDH4ppNo2,0.000000e+00,-3.859694e-10
HPYRRyNo2,0.000000e+00,-4.536509e-10
arm_HPYRRy,0.000000e+00,-4.536509e-10
HPYRRyNo1,0.000000e+00,-4.536509e-10


In [40]:
enzyme_fva[enzyme_fva.index.str.startswith("prot_")].sort_values("maximum", ascending=False)

,minimum,maximum
prot_P37769_exchange,0.000000e+00,2.314800e+01
prot_P0AC16_exchange,6.629624e-07,1.248440e+01
prot_P0AEK2_exchange,1.163420e-05,7.532970e+00
prot_P0AA84_exchange,0.000000e+00,2.924040e+00
prot_P21151_exchange,1.244089e-04,2.031869e+00
...,...,...
prot_P69330_exchange,0.000000e+00,-1.542749e-11
prot_P0A9I1_exchange,0.000000e+00,-1.543561e-11
prot_P38135_exchange,0.000000e+00,-1.958814e-11
prot_P0A746_exchange,0.000000e+00,-3.368114e-10


## Exercise

* Identify enzymatic bottlenecks in the enzymed constrained model in **Acetate** as carbon source (shadow prices?).
* Prepare and limit the model for the medium with Glucose
* Does the model grow? If not, try flexibilizing the model.
* Identify enzymatic bottlenecks in the enzymed constrained model in with **Glucose** as carbon source (shadow prices?).
